In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [2]:
# parameter 설정

lr = 1e-3
batch_size = 64
num_epoch = 10

data_dir = './drive/My Drive/Colab Notebooks/Pytorch_Unet/datasets'
ckpt_dir = './drive/My Drive/Colab Notebooks/Pytorch_Unet/checkpoint'     # 체크 포인트. 주기마다 모델 저장
log_dir = './drive/My Drive/Colab Notebooks/Pytorch_Unet/log'             # 텐서 보드 확인을 위한 로그 저장

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # CUDA 디바이스 설정


In [ ]:
# Networt 구축

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):    # Conv, B-norm, ReLU
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)        # 하나의 function으로 정의

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=1, out_channels=64)         # predefine이므로 나머지는 작성하지 않는다
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)        # 인코더의 첫번째 파트

        self.pool1 = nn.MaxPool2d(kernel_size=2)                    # 빨간색 화살표

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)     # 마지막 중간 Encoder 파트


        # 오른쪽 Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)     # Encoder와의 명명 매치

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,                # Up-Conv 구현
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)  # Enc 출력이 붙으므로 (* 2)가 붙는다. 
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)      # 녹색 화살표. 1 x 1

    def forward(self, x):                               # 각각의 layer 연결
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)

        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)      # cat 함수를 이용하여 Enc와 UpConv 합
                                                        # dim = [0:batch, 1:channel, 2:height, 3:width]
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x

In [ ]:
# Dataloader 직접 구현해보기

class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_data = os.listdir(self.data_dir)        # data_dir 내 모든 파일을 불러온다

        lst_label = [f for f in lst_data if f.startswith('label')]      # label 정렬, startswitch를 통해 원하는 파일 정렬
        lst_input = [f for f in lst_data if f.startswith('input')]      # input 정렬

        lst_label.sort()                            # sort
        lst_input.sort()

        self.lst_label = lst_label                  # 클래스의 parameter로 가져온다
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_label)                  # 함수의 length 확인 함수

    def __getitem__(self, index):                   # 실제 데이터 get. index를 입력받아 index 해당 파일 load
        label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        label = label/255.0                         # 0 ~ 1 사이로 normalization
        input = input/255.0

        if label.ndim == 2:                         # x, y, channel axis 임의로 생성
            label = label[:, :, np.newaxis]
        if input.ndim == 2:
            input = input[:, :, np.newaxis]

        data = {'input': input, 'label': label}     # dict 형태로 output한다

        if self.transform:                          # transform 함수가 정의되어 있다면 이를 return
            data = self.transform(data)

        return data

In [ ]:
# 네트워크 저장 함수

def save(ckpt_dir, net, optim, epoch):        # check point 저장
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)                 # 없다면 체크 포인트 생성

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},    # net, optim -> pth로 저장
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))              # net : , optim : 

def load(ckpt_dir, net, optim):               # check point 로드
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()                           # check point list 한 후 마지막 사용을 위해 sort -> 1, 10도 sort가 되는지 확인

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))         # 마지막 sort를 사용. dict_model에 넣어준다

    net.load_state_dict(dict_model['net'])                                # net, optim 모델에서 로드
    optim.load_state_dict(dict_model['optim'])

    return net, optim

In [ ]:
# MNIST 데이터 불러오기

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])      # 텐서 바꾸기 + Normalization = transform

dataset = datasets.MNIST(download=True, root='./drive/My Drive/Colab Notebooks/MNIST_Test', train=True, transform=transform)  # 데이터 셋 설정. transform으로 데이터 수정
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)                             # 함수를 이용해 데이터 로드. shuffle, num_worker는 추가 가능

num_data = len(loader.dataset)                      # dataset 개수 확인
num_batch = np.ceil(num_data / batch_size)          # dataset 개수에서 batch_size를 나눠 training batch 개수 구함

In [ ]:
# 네트워크 설정, 손실함수 구현

net = Net().to(device)                              # 앞서 설정한 Net 구조 사용 변수
params = net.parameters()                           # Net의 변수 저장

fn_loss = nn.CrossEntropyLoss().to(device)          # loss 함수. CrossEntropyLoss 사용
fn_pred = lambda output: torch.softmax(output, dim=1)         # softmax 사용 predict 모델 생성 변수
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()           # 예측 모델과 실제 모델을 합쳐 정확성 체크

optim = torch.optim.Adam(params, lr=lr)             # Adam 사용. Adam 논문 정독 필요

writer = SummaryWriter(log_dir=log_dir)             # log 저장

In [ ]:
# 트레이닝

for epoch in range(1, num_epoch + 1):               # 한 epoch 학습
    net.train()

    loss_arr = []                                   # Loss 함수 저장
    acc_arr = []                                    # 정확도 함수 저장

    for  batch, (input, label) in enumerate(loader, 1):     # batch 마다 진행
        input = input.to(device)
        label = label.to(device)

        output = net(input)                         # Net에 input
        pred = fn_pred(output)                      # 구한 output softmax

        optim.zero_grad()                           # 역전파 단계를 실행하기 전에 변화도를 0으로 설정

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        loss.backward()                             # 역전파 : 모델의 매개변수에 대한 손실의 변화도를 계산

        optim.step()                                # Optimizer의 step 함수를 호출 후 매개변수가 갱신

        loss_arr += [loss.item()]                   # Tensorboard 확인을 위한 loss 저장
        acc_arr += [acc.item()]                     # Tensorboard 확인을 위한 acc 저장

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    writer.add_scalar('loss', np.mean(loss_arr), epoch)         # Tensorboard loss 입력
    writer.add_scalar('acc', np.mean(acc_arr), epoch)           # Tensorboard acc 입력

    save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)  # save

writer.close()          # 학습 마친 이후 writer 종료

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
TRAIN: EPOCH 0005/0010 | BATCH 0630/0938 | LOSS: 0.1782 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0631/0938 | LOSS: 0.1781 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0632/0938 | LOSS: 0.1782 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0633/0938 | LOSS: 0.1782 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0634/0938 | LOSS: 0.1783 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0635/0938 | LOSS: 0.1783 | ACC 0.9488
TRAIN: EPOCH 0005/0010 | BATCH 0636/0938 | LOSS: 0.1782 | ACC 0.9488
TRAIN: EPOCH 0005/0010 | BATCH 0637/0938 | LOSS: 0.1780 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0638/0938 | LOSS: 0.1780 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0639/0938 | LOSS: 0.1781 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0640/0938 | LOSS: 0.1782 | ACC 0.9489
TRAIN: EPOCH 0005/0010 | BATCH 0641/0938 | LOSS: 0.1781 | ACC 0.9488
TRAIN: EPOCH 0005/0010 | BATCH 0642/0938 | LOSS: 0.1781 | ACC 0.9488
TRAIN: EPOCH 0005/0010 | BATCH 0643/0938 | LOSS: 0.1780 | ACC 0.948